In [3]:
!pip install folium
!pip install branca

import folium
import pandas as pd
import numpy as np

from folium.plugins import MarkerCluster


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 5.0 MB/s eta 0:00:00


In [5]:
spacex_geo_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df = pd.read_csv(spacex_geo_url)
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [7]:
## Task 1: Mark all launch sites on a map


In [9]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)


In [11]:
launch_sites = spacex_df.groupby('Launch Site').first().reset_index()

for i, row in launch_sites.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=row['Launch Site'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(site_map)

site_map


In [13]:
# Task 2: Mark the success/failed launches for each site on the map

In [15]:
from folium.plugins import MarkerCluster

marker_cluster = MarkerCluster().add_to(site_map)

In [17]:
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

In [19]:
for i, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.Icon(color=row['marker_color']),
        popup=f"{row['Launch Site']} | Success: {row['class']}"
    ).add_to(marker_cluster)

In [21]:
site_map


In [23]:
# TASK 3: Calculate the distances between a launch site to its proximities


In [25]:
from math import radians, cos, sin, asin, sqrt

def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    distance = R * c
    return distance


In [27]:
distance_coast = calculate_distance(28.563197, -80.576820, 28.563, -80.567)
print(f"Distance to coastline: {distance_coast:.2f} km")


Distance to coastline: 0.96 km


In [29]:
folium.Marker(
    [28.563, -80.567],
    icon=folium.Icon(color='orange'),
    popup=f"Coastline<br>Distance: {distance_coast:.2f} km"
).add_to(site_map)


In [31]:
folium.PolyLine(
    locations=[(28.563197, -80.576820), (28.563, -80.567)],
    color='blue'
).add_to(site_map)


In [33]:
site_map
